In [10]:
# Set seed for reproducibility
seed = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Print TensorFlow version
print(tf.__version__)

# Import other libraries
import requests
from io import BytesIO
import cv2
from PIL import Image
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

2.18.0


In [ ]:
densenet = tfk.applications.DenseNet121(
    input_shape=(128, 128, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg',
)
# Display a summary of the model architecture
densenet.summary(expand_nested=True)

# Display model architecture with layer shapes and trainable parameters
tfk.utils.plot_model(densenet, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

# Train the DensNet121 with Oxford pet

In [12]:
def load_oxford_pets():
    # Load dataset with TensorFlow Datasets, obtaining dataset info
    dataset, info = tfds.load(
        'oxford_iiit_pet',
        with_info=True,
        as_supervised=True
    )

    # Split dataset into training and test sets
    train_dataset, test_dataset = dataset['train'], dataset['test']

    # Define image preprocessing function
    def preprocess(image, label):
        image = tf.cast(image, tf.float32)
        height = tf.shape(image)[0]
        width = tf.shape(image)[1]
        crop_size = tf.minimum(height, width)
        height_offset = (height - crop_size) // 2
        width_offset = (width - crop_size) // 2

        # Centre-crop and resize image, normalising pixel values
        image = tf.image.crop_to_bounding_box(
            image,
            height_offset,
            width_offset,
            crop_size,
            crop_size
        )
        image = tf.image.resize(image, [128, 128])
        image = image / 255.0
        return image, label

    # Initialise NumPy arrays for training and test sets
    n_train = info.splits['train'].num_examples
    n_test = info.splits['test'].num_examples
    img_shape = (128, 128, 3)

    X_train = np.zeros((n_train, *img_shape), dtype=np.float32)
    y_train = np.zeros(n_train, dtype=np.int64)
    X_test = np.zeros((n_test, *img_shape), dtype=np.float32)
    y_test = np.zeros(n_test, dtype=np.int64)

    # Batch-process and store training data
    train_dataset = train_dataset.map(preprocess).batch(32)
    idx = 0
    for batch_images, batch_labels in train_dataset:
        batch_size = batch_images.shape[0]
        X_train[idx:idx + batch_size] = batch_images.numpy()
        y_train[idx:idx + batch_size] = batch_labels.numpy()
        idx += batch_size

        # Optional: print progress for training data
        if idx % 500 == 0:
            print(f"Processed {idx}/{n_train} training images")

    # Batch-process and store test data
    test_dataset = test_dataset.map(preprocess).batch(32)
    idx = 0
    for batch_images, batch_labels in test_dataset:
        batch_size = batch_images.shape[0]
        X_test[idx:idx + batch_size] = batch_images.numpy()
        y_test[idx:idx + batch_size] = batch_labels.numpy()
        idx += batch_size

        # Optional: print progress for test data
        if idx % 500 == 0:
            print(f"Processed {idx}/{n_test} test images")

    # Retrieve and format class names
    class_names = list(map(str.lower, info.features['label'].names))
    return (X_train, y_train), (X_test, y_test), class_names

# Execute function and load data
(X_train, y_train), (X_test, y_test), class_names = load_oxford_pets()

# Display data shapes for training and test sets
print("Training set shape (images):", X_train.shape)
print("Training set shape (labels):", y_train.shape)
print("Test set shape (images):", X_test.shape)
print("Test set shape (labels):", y_test.shape)

Training set shape (images): (3680, 128, 128, 3)
Training set shape (labels): (3680,)
Test set shape (images): (3669, 128, 128, 3)
Test set shape (labels): (3669,)


In [13]:
# Split test set into validation and test sets with stratification
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, random_state=seed, test_size=0.5, stratify=y_test)

# Convert class labels to categorical format for training, validation, and test sets
y_train = tfk.utils.to_categorical(y_train, num_classes=len(class_names))
y_val = tfk.utils.to_categorical(y_val, num_classes=len(class_names))
y_test = tfk.utils.to_categorical(y_test, num_classes=len(class_names))

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (3680, 128, 128, 3), y_train shape: (3680, 37)
X_val shape: (1834, 128, 128, 3), y_val shape: (1834, 37)
X_test shape: (1835, 128, 128, 3), y_test shape: (1835, 37)


In [ ]:
# Freeze all layers in MobileNetV3Small to use it solely as a feature extractor
densenet.trainable = False

# Define input layer with shape matching the input images
inputs = tfk.Input(shape=(128, 128, 3), name='input_layer')

# Apply data augmentation for training robustness
augmentation = tf.keras.Sequential([
    tfkl.RandomFlip("horizontal"),
    tfkl.RandomTranslation(0.2, 0.2)
], name='preprocessing')

x = augmentation(inputs)

# Pass augmented inputs through the MobileNetV3Small feature extractor
x = densenet(x)

# Add a dropout layer for regularisation
x = tfkl.Dropout(0.3, name='dropout')(x)

# Add final Dense layer for classification with softmax activation
outputs = tfkl.Dense(y_train.shape[-1], activation='softmax', name='dense')(x)

# Define the complete model linking input and output
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model with categorical cross-entropy loss and Adam optimiser
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

# Display a summary of the model architecture
tl_model.summary(expand_nested=True)

# Display model architecture with layer shapes and trainable parameters
tfk.utils.plot_model(tl_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [15]:
# Train the model
history = tl_model.fit(
    x = X_train*255,
    y = y_train,
    batch_size = 64,
    epochs = 200,
    validation_data = (X_val*255, y_val),
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

# Calculate and print the final validation accuracy
final_val_accuracy = round(max(history['val_accuracy'])* 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file with the accuracy included in the filename
model_filename = 'Pets_DenseNet121_'+str(final_val_accuracy)+'.keras'
tl_model.save(model_filename)

# Delete the model to free up resources
del tl_model

Epoch 1/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.0332 - loss: 13.3717 - val_accuracy: 0.0671 - val_loss: 6.6948
Epoch 2/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.0737 - loss: 9.2935 - val_accuracy: 0.0932 - val_loss: 5.6202
Epoch 3/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.0806 - loss: 7.6382 - val_accuracy: 0.1292 - val_loss: 5.1306
Epoch 4/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.1007 - loss: 6.5995 - val_accuracy: 0.1450 - val_loss: 4.9547
Epoch 5/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.1212 - loss: 5.8093 - val_accuracy: 0.1538 - val_loss: 4.6036
Epoch 6/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.1499 - loss: 5.1926 - val_accuracy: 0.1778 - val_loss: 4.2429
Epoch 7/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.1510 - loss: 4.6869 - val_accuracy: 0.1919 - val_loss: 4.1716
Epoch 8/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.1703 - loss: 4.3331 - val_accuracy: 0.1832 - 

KeyboardInterrupt: 